In [1]:
import numpy as np
import pandas as pd

In [2]:
#!pip install openpyxl

In [3]:
#pip install pandas --upgrade

In [85]:
# Function of depression_TRD markov train calculation. 
# Inputs: inc: initial incidence; p_m: probabitly matirx (study period * 16)
def Marcov_depres_TRD(inc, p_m, period): 
    results = []
    results.append([inc,0,0,0,0,0]) 
    for t in range(period):
        p_m_t = p_m[t,:]
        t_m = [
                # NTRD      NTRD-c     TRD        TRD-c      DEAD       LISU
                [ p_m_t[4], p_m_t[1],  p_m_t[0],  0,         p_m_t[2],  p_m_t[3] ], #NTRD
                [ 0,        p_m_t[12], p_m_t[9],  0,         p_m_t[10], p_m_t[11]], #NTRD-c
                [ 0,        0,         p_m_t[8],  p_m_t[5],  p_m_t[6],  p_m_t[7] ], #TRD
                [ 0,        0,         0,         p_m_t[15], p_m_t[13], p_m_t[14]], #TRD-c
                [ 0,        0,         0,         0,         1,         0        ], #DEAD
                [ 0,        0,         0,         0,         0,         1        ]  #LISU
            ]
        results.append(np.dot(results[t], t_m)) 

    return results

In [93]:
a = range(period)

For baseline + COVID-19 

In [110]:
# Define the file path, sheet name, and matrix location
file_path = "G:\我的云端硬盘\Model_24May\Closed and open cohort_Markov model_2_subgroup.xlsx"
sheet_name = "TP_BC"
data_p = pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl')
sheet_name = "Model input"
data_i = pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl')
sheet_name = "Cost_DPSA"
data_c = pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl')


Subgroup weights: 
w10_24_F_No_history:0; w25_40_F_No_history:1; w40_65_F_No_history:2; w65_F_No_history:3; w10_24_F_History:4; w25_40_F_History:5; w40_65_F_History:6; w65_F_History:7; w10_24_M_No_history:8; w25_40_M_No_history:9;w40_65_M_No_history:10; w65_M_No_history:11; w10_24_M_History:12; w25_40_M_History:13; w40_65_M_History:14; w65_M_History:15;

In [87]:
weights = data_i.iloc[52:68, 3].to_numpy()

Incidence list

In [88]:
inc = data_i.iloc[27:47, 3].to_numpy()

Period: length of the projection period: 20

In [134]:
period = 20

All-cause cost list

In [175]:
cost = data_c.iloc[0:64, 7].to_numpy()
c_m_pre = cost.reshape(4,16)
c_m = []
c_m = c_m_pre[0,:]
c_m = np.vstack((c_m, c_m_pre[2,:]))
c_m = np.vstack((c_m, c_m_pre[1,:]))
c_m = np.vstack((c_m, c_m_pre[3,:]))
zero_rows = np.zeros((2, c_m.shape[1]))
c_m = np.vstack((c_m, zero_rows))
#c_m = c_m.transpose() # the cost on the stages in different weight index: weight index 16 * stages number 6

3D matrix for all probability matrices, listing according to weight index

In [176]:
p_m_all = []
p_m_all.append(data_p.iloc[8:28, 1:17].to_numpy()) # w10_24_F_No_history: 0
p_m_all.append(data_p.iloc[36:56, 1:17].to_numpy()) # w25_40_F_No_history: 1
p_m_all.append(data_p.iloc[64:84, 1:17].to_numpy()) # w40_65_F_No_history: 2
p_m_all.append(data_p.iloc[92:112, 1:17].to_numpy()) # w65_F_No_history: 3
p_m_all.append(data_p.iloc[120:140, 1:17].to_numpy()) # w10_24_F_History: 4
p_m_all.append(data_p.iloc[148:168, 1:17].to_numpy()) # w25_40_F_History: 5
p_m_all.append(data_p.iloc[176:196, 1:17].to_numpy()) # w40_65_F_History: 6
p_m_all.append(data_p.iloc[204:224, 1:17].to_numpy()) # w65_F_History: 7
p_m_all.append(data_p.iloc[232:252, 1:17].to_numpy()) # w10_24_M_No_history: 8
p_m_all.append(data_p.iloc[260:280, 1:17].to_numpy()) # w25_40_M_No_history: 9
p_m_all.append(data_p.iloc[288:308, 1:17].to_numpy()) # w40_65_M_No_history: 10
p_m_all.append(data_p.iloc[316:336, 1:17].to_numpy()) # w65_M_No_history: 11
p_m_all.append(data_p.iloc[344:364, 1:17].to_numpy()) # w10_24_M_History: 12
p_m_all.append(data_p.iloc[372:392, 1:17].to_numpy()) # w25_40_M_History: 13
p_m_all.append(data_p.iloc[400:420, 1:17].to_numpy()) # w40_65_M_History: 14
p_m_all.append(data_p.iloc[428:448, 1:17].to_numpy()) # w65_M_History: 15


Initialize the 3D results matrix:  listing len(inc) matrices of projecstion of different year, from (2014 + current index) to (2014 + current index + period)  

In [177]:
results_2014 = []

1. For 2014 cohort (incidence index 0)

In [178]:
results_2014 = []
cost_2014 =[]

for i in range(len(weights)):
    inc_n = inc[0] * weights[i] # incidence index 0 * weight index i 
    p_m = p_m_all[i] # weight index i
    result = np.array(Marcov_depres_TRD(inc_n, p_m, period))
    results_2014.append(result)
    cost_2014.append(np.dot(result,c_m[:,i]))

1.1 Cost of subgroup: Age: 10-24, Female, Medical history: negative. 20 year prediction from 2014

In [184]:
cost_2014[0][1:]


array([4362760.75119983, 4070002.40103566, 3817078.76771778,
       3600712.03164576, 3410648.65125495, 3240285.13927879,
       3085589.01520228, 2943990.61724979, 2813738.69697226,
       2693549.37416939, 2579238.52109123, 2470543.80556102,
       2367203.0294545 , 2268956.86447882, 2175550.98756621,
       2086737.71694479, 2002277.23452044, 1921938.4670263 ,
       1845499.68714931, 1772748.88624947])

1.2 Cost of subgroup: Age: 25-40, Female, Medical history: negative. 20 year prediction from 2014

In [185]:
cost_2014[1][1:]

array([11975887.69838755, 11445155.78632115, 10924277.26438902,
       10448076.69377037, 10010672.26871812,  9604811.31047555,
        9225230.06155874,  8868343.50137054,  8531679.54628933,
        8213464.61885009,  7907978.08093514,  7614948.79349069,
        7334058.12355612,  7064951.36868005,  6807247.39367401,
        6560546.7232678 ,  6324438.30320057,  6098505.1150558 ,
        5882328.80627087,  5675493.47582212])

1.3 Total cost of whole population. 20 year prediction from 2014

In [188]:
cost_2014_whole_po =  np.sum(cost_2014, axis=0)
cost_2014_whole_po[1:]

array([2.91616661e+08, 2.74152186e+08, 2.56718019e+08, 2.41482664e+08,
       2.28272541e+08, 2.16699777e+08, 2.06433502e+08, 1.97222072e+08,
       1.88877525e+08, 1.81258371e+08, 1.73968060e+08, 1.66996600e+08,
       1.60333144e+08, 1.53966287e+08, 1.47884296e+08, 1.42075305e+08,
       1.36527461e+08, 1.31229046e+08, 1.26168564e+08, 1.21334818e+08])

2. For 2015 - 2022, 20 years cot prediction 

2015

In [189]:
results_2015 = []
cost_2015 =[]

for i in range(len(weights)):
    inc_n = inc[1] * weights[i] # incidence index 1 * weight index i 
    p_m = p_m_all[i] # weight index i
    result = np.array(Marcov_depres_TRD(inc_n, p_m, period))
    results_2015.append(result)
    cost_2015.append(np.dot(result,c_m[:,i]))

cost_2015_whole_po =  np.sum(cost_2015, axis=0)
cost_2015_whole_po[1:]

array([2.94225221e+08, 2.76604524e+08, 2.59014404e+08, 2.43642766e+08,
       2.30314477e+08, 2.18638193e+08, 2.08280084e+08, 1.98986257e+08,
       1.90567066e+08, 1.82879757e+08, 1.75524234e+08, 1.68490412e+08,
       1.61767351e+08, 1.55343541e+08, 1.49207146e+08, 1.43346192e+08,
       1.37748722e+08, 1.32402911e+08, 1.27297163e+08, 1.22420179e+08])

2016

In [191]:
results_2016 = []
cost_2016 =[]

for i in range(len(weights)):
    inc_n = inc[2] * weights[i] # incidence index 2 * weight index i 
    p_m = p_m_all[i] # weight index i
    result = np.array(Marcov_depres_TRD(inc_n, p_m, period))
    results_2016.append(result)
    cost_2016.append(np.dot(result,c_m[:,i]))

cost_2016_whole_po =  np.sum(cost_2016, axis=0)
cost_2016_whole_po[1:]

array([2.83011802e+08, 2.66062659e+08, 2.49142928e+08, 2.34357130e+08,
       2.21536804e+08, 2.10305522e+08, 2.00342179e+08, 1.91402555e+08,
       1.83304234e+08, 1.75909901e+08, 1.68834709e+08, 1.62068959e+08,
       1.55602124e+08, 1.49423137e+08, 1.43520610e+08, 1.37883027e+08,
       1.32498886e+08, 1.27356813e+08, 1.22445654e+08, 1.17754539e+08])

2017

In [193]:
results_2017 = []
cost_2017 =[]

for i in range(len(weights)):
    inc_n = inc[3] * weights[i] # incidence index 3 * weight index i 
    p_m = p_m_all[i] # weight index i
    result = np.array(Marcov_depres_TRD(inc_n, p_m, period))
    results_2017.append(result)
    cost_2017.append(np.dot(result,c_m[:,i]))

cost_2017_whole_po =  np.sum(cost_2017, axis=0)
cost_2017_whole_po[1:]

array([2.91006868e+08, 2.73578913e+08, 2.56181201e+08, 2.40977704e+08,
       2.27795205e+08, 2.16246641e+08, 2.06001833e+08, 1.96809665e+08,
       1.88482567e+08, 1.80879345e+08, 1.73604279e+08, 1.66647397e+08,
       1.59997875e+08, 1.53644331e+08, 1.47575059e+08, 1.41778214e+08,
       1.36241972e+08, 1.30954635e+08, 1.25904736e+08, 1.21081098e+08])

2018

In [194]:
results_2018 = []
cost_2018 =[]

for i in range(len(weights)):
    inc_n = inc[4] * weights[i] # incidence index 4 * weight index i 
    p_m = p_m_all[i] # weight index i
    result = np.array(Marcov_depres_TRD(inc_n, p_m, period))
    results_2018.append(result)
    cost_2018.append(np.dot(result,c_m[:,i]))

cost_2018_whole_po =  np.sum(cost_2018, axis=0)
cost_2018_whole_po[1:]

array([2.85518729e+08, 2.68419450e+08, 2.51349844e+08, 2.36433073e+08,
       2.23499184e+08, 2.12168415e+08, 2.02116816e+08, 1.93098005e+08,
       1.84927948e+08, 1.77468117e+08, 1.70330252e+08, 1.63504571e+08,
       1.56980453e+08, 1.50746732e+08, 1.44791920e+08, 1.39104399e+08,
       1.33672565e+08, 1.28484944e+08, 1.23530281e+08, 1.18797613e+08])

2019

In [195]:
results_2019 = []
cost_2019 =[]

for i in range(len(weights)):
    inc_n = inc[5] * weights[i] # incidence index 5 * weight index i 
    p_m = p_m_all[i] # weight index i
    result = np.array(Marcov_depres_TRD(inc_n, p_m, period))
    results_2019.append(result)
    cost_2019.append(np.dot(result,c_m[:,i]))

cost_2019_whole_po =  np.sum(cost_2019, axis=0)
cost_2019_whole_po[1:]

array([2.88059534e+08, 2.70808090e+08, 2.53586584e+08, 2.38537069e+08,
       2.25488083e+08, 2.14056483e+08, 2.03915435e+08, 1.94816366e+08,
       1.86573605e+08, 1.79047389e+08, 1.71846006e+08, 1.64959583e+08,
       1.58377408e+08, 1.52088213e+08, 1.46080410e+08, 1.40342277e+08,
       1.34862105e+08, 1.29628320e+08, 1.24629566e+08, 1.19854782e+08])

2020

In [196]:
results_2020 = []
cost_2020 =[]

for i in range(len(weights)):
    inc_n = inc[6] * weights[i] # incidence index 6 * weight index i 
    p_m = p_m_all[i] # weight index i
    result = np.array(Marcov_depres_TRD(inc_n, p_m, period))
    results_2020.append(result)
    cost_2020.append(np.dot(result,c_m[:,i]))

cost_2020_whole_po =  np.sum(cost_2020, axis=0)
cost_2020_whole_po[1:]

array([3.24071211e+08, 3.04663083e+08, 2.85288634e+08, 2.68357709e+08,
       2.53677408e+08, 2.40816690e+08, 2.29407862e+08, 2.19171276e+08,
       2.09898049e+08, 2.01430945e+08, 1.93329282e+08, 1.85581956e+08,
       1.78176912e+08, 1.71101475e+08, 1.64342609e+08, 1.57887125e+08,
       1.51721851e+08, 1.45833765e+08, 1.40210094e+08, 1.34838391e+08])

2021

In [197]:
results_2021 = []
cost_2021 =[]

for i in range(len(weights)):
    inc_n = inc[7] * weights[i] # incidence index 7 * weight index i 
    p_m = p_m_all[i] # weight index i
    result = np.array(Marcov_depres_TRD(inc_n, p_m, period))
    results_2021.append(result)
    cost_2021.append(np.dot(result,c_m[:,i]))

cost_2021_whole_po =  np.sum(cost_2021, axis=0)
cost_2021_whole_po[1:]

array([3.58456772e+08, 3.36989345e+08, 3.15559172e+08, 2.96831792e+08,
       2.80593838e+08, 2.66368534e+08, 2.53749173e+08, 2.42426434e+08,
       2.32169272e+08, 2.22803766e+08, 2.13842477e+08, 2.05273121e+08,
       1.97082365e+08, 1.89256190e+08, 1.81780174e+08, 1.74639731e+08,
       1.67820291e+08, 1.61307450e+08, 1.55087080e+08, 1.49145413e+08])

In [198]:
results_2022 = []
cost_2022 =[]

for i in range(len(weights)):
    inc_n = inc[8] * weights[i] # incidence index 8 * weight index i 
    p_m = p_m_all[i] # weight index i
    result = np.array(Marcov_depres_TRD(inc_n, p_m, period))
    results_2022.append(result)
    cost_2022.append(np.dot(result,c_m[:,i]))

cost_2022_whole_po =  np.sum(cost_2022, axis=0)
cost_2022_whole_po[1:]

array([3.18989601e+08, 2.99885802e+08, 2.80815156e+08, 2.64149717e+08,
       2.49699610e+08, 2.37040555e+08, 2.25810624e+08, 2.15734553e+08,
       2.06606735e+08, 1.98272400e+08, 1.90297776e+08, 1.82671931e+08,
       1.75383003e+08, 1.68418513e+08, 1.61765629e+08, 1.55411370e+08,
       1.49342771e+08, 1.43547014e+08, 1.38011525e+08, 1.32724053e+08])